In [1]:
import numpy as np
import pandas as pd
import pandas as pd
from collections import Counter, defaultdict
import re
import ast

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
picks = pd.read_parquet('../../Data/2023_2025_pick_trades.parquet')
# Normalize franchise ownership
OWNER_REMAP = {
    "Weeks": "Gio"
}

picks = picks.copy()

picks["receiving_owner"] = picks["receiving_owner"].replace(OWNER_REMAP)
picks["giving_owner"] = picks["giving_owner"].replace(OWNER_REMAP)


In [3]:
def safe_parse_pick_list(x):
    """
    Robustly parse pick lists that may be:
    - Python lists
    - numpy arrays
    - empty lists
    - NaN
    - string representations like "['2024_1','2024_2']"
    """
    # Case 1: list / tuple / set / numpy array
    if isinstance(x, (list, tuple, set, np.ndarray)):
        return [str(i) for i in x if i not in (None, "", [])]

    # Case 2: NaN / None
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []

    # Case 3: string
    if isinstance(x, str):
        x = x.strip()
        if x == "" or x == "[]":
            return []

        # Try parsing string list safely
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, (list, tuple, set)):
                return [str(i) for i in parsed if i not in (None, "", [])]
        except Exception:
            # fallback: comma split
            return [i.strip().strip("'\"") for i in x.split(",") if i.strip()]

    # Case 4: single scalar (very rare but safe)
    return [str(x)]


In [4]:
def counter_from_list(lst):
    c = Counter()
    for p in lst:
        if p and isinstance(p, str):
            c[p] += 1
    return c

In [5]:
if 'date' in picks.columns:
    picks['date'] = pd.to_datetime(picks['date'], errors='coerce')

# normalize week strings ordering: pre-season, '1'..'14', offseason
WEEKS = [
    'pre-season',
    '1','2','3','4','5','6','7','8','9','10','11','12','13','14',
    'offseason'
]

week_order = {w: i for i, w in enumerate(WEEKS)}


In [6]:
trade_effects = []

for _, row in picks.iterrows():
    season = int(row['season'])
    week = row['week']  # should be 'pre-season', 'offseason', or '1'..'14'
    ro = row['receiving_owner']
    go = row['giving_owner']

    # parse lists robustly
    ro_recv = safe_parse_pick_list(row.get('ro_picks_received', []))
    ro_gave = safe_parse_pick_list(row.get('ro_picks_gave', []))
    go_recv = safe_parse_pick_list(row.get('go_picks_received', []))
    go_gave = safe_parse_pick_list(row.get('go_picks_gave', []))

    # receiving owner's delta = received - gave
    ro_delta = counter_from_list(ro_recv)
    for p in counter_from_list(ro_gave):
        ro_delta[p] -= counter_from_list(ro_gave)[p]
    # alternative (clean): subtract counts
    # but above double-computes counter_from_list(ro_gave) - so simpler below:
    ro_delta = counter_from_list(ro_recv)
    for p,cnt in counter_from_list(ro_gave).items():
        ro_delta[p] -= cnt

    # giving owner's delta = received - gave
    go_delta = counter_from_list(go_recv)
    for p,cnt in counter_from_list(go_gave).items():
        go_delta[p] -= cnt

    # append only if non-empty delta (helps speed / clarity)
    if sum(ro_delta.values()) != 0:
        trade_effects.append({
            'owner': ro,
            'season': season,
            'week': week,
            'delta': ro_delta
        })
    if sum(go_delta.values()) != 0:
        trade_effects.append({
            'owner': go,
            'season': season,
            'week': week,
            'delta': go_delta
        })

trade_effects_df = pd.DataFrame(trade_effects)

In [7]:
# quick helper to group deltas by (owner, season, week)
from collections import defaultdict
deltas_by_owner_season_week = defaultdict(list)
for _, r in trade_effects_df.iterrows():
    key = (r['owner'], int(r['season']), r['week'])
    deltas_by_owner_season_week[key].append(r['delta'])

In [8]:
ROUNDS = [1,2,3,4]

def baseline_picks_for_season(season):
    """
    Return Counter baseline at START of season for each owner:
    - season 2023 baseline: 2024,2025,2026 (the league inception)
    - season 2024 baseline: 2025,2026,2027
    - season 2025 baseline: 2026,2027,2028
    (adjust if you add more seasons)
    """
    years = []
    if season == 2023:
        years = [2024,2025,2026]
    elif season == 2024:
        years = [2025,2026,2027]
    elif season == 2025:
        years = [2026,2027,2028]
    else:
        # default: rolling forward by offset
        years = [season+1, season+2, season+3]

    c = Counter()
    for y in years:
        for r in ROUNDS:
            c[f"{y}_{r}"] = 1
    return c

In [9]:
owners = sorted(set(picks['receiving_owner']).union(set(picks['giving_owner'])))
pick_years = [2024,2025,2026,2027,2028]  # expand if needed
pick_cols = [f"{y}_{r}" for y in pick_years for r in ROUNDS]

In [10]:
records = []

for season in sorted(picks['season'].unique()):
    season = int(season)
    for owner in owners:
        # start baseline at start of season
        current_picks = baseline_picks_for_season(season)

        # For consistency, snapshot before applying any trades that week:
        for week in WEEKS:
            # apply all deltas that occurred exactly in this (owner,season,week)
            key = (owner, season, week)
            if key in deltas_by_owner_season_week:
                for delta in deltas_by_owner_season_week[key]:
                    # delta is a Counter-like dict
                    for pick, val in delta.items():
                        current_picks[pick] += val

            # record snapshot after applying this week's trades
            row = {
                'owner': owner,
                'season': season,
                'week': week
            }
            for col in pick_cols:
                row[col] = int(current_picks.get(col, 0))
            records.append(row)

In [11]:
# finalize dataframe
pick_time_series_clean = pd.DataFrame(records)
#pick_time_series_clean

In [12]:
pick_time_series_clean['week'] = pick_time_series_clean['week'].astype(str)

pick_time_series_clean['week_sort'] = (
    pick_time_series_clean['week']
    .map(week_order)
)

In [13]:
pick_time_series_clean = (
    pick_time_series_clean
    .sort_values(
        by=['owner', 'season', 'week_sort'],
        ascending=[True, True, True]
    )
    .drop(columns='week_sort')
    .reset_index(drop=True)
)

In [14]:
pick_time_series_clean

,owner,season,week,2024_1,2024_2,2024_3,2024_4,2025_1,2025_2,2025_3,2025_4,2026_1,2026_2,2026_3,2026_4,2027_1,2027_2,2027_3,2027_4,2028_1,2028_2,2028_3,2028_4
0,Brigido,2023,pre-season,1,1,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
1,Brigido,2023,1,1,1,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
2,Brigido,2023,2,1,2,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
3,Brigido,2023,3,1,3,3,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
4,Brigido,2023,4,1,3,3,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
5,Brigido,2023,5,0,2,3,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
6,Brigido,2023,6,0,2,3,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
7,Brigido,2023,7,1,2,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
8,Brigido,2023,8,1,3,1,1,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
9,Brigido,2023,9,1,1,0,1,2,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0
